found: concorde concorde.c concorde.o 


In [22]:




dist_pts <- function(pt1,pt2){
        return(sqrt((pt1[[1]]-pt2[[1]])^2 + (pt1[[2]]-pt2[[2]])^2))
}

angle <- function(x,y){
  dot.prod <- x%*%y 
  norm.x <- norm(x,type="2")
  norm.y <- norm(y,type="2")
  theta <- acos(dot.prod / (norm.x * norm.y))
  as.numeric(theta)
}

angle_pts <- function(pt1,pt2){
    
    if (pt1[[2]] -pt2[[2]] == 0){
        if (pt1[[1]] -pt2[[1]]>0){
            return(pi/2)
        }
        if (pt1[[1]] -pt2[[1]]<0){
            return(-pi/2)
        }
        if (pt1[[1]] -pt2[[1]]==0){
            return(0)
        }
    } else{
        x<- pt1[[1]] -pt2[[1]]
        y <-pt1[[2]] -pt2[[2]]
        angle <- atan((y)/(x))
        if (x < 0 & y>0){
            angle <- pi + angle
        }
        if (x >0 & y <0){
            angle <-2*pi +angle
        }
        if (x < 0 & y<0){
            angle <- pi + angle
        }
        return(angle)
        }
}


get_item <- function(item_to_get, database){
    counter <-1
    cont <- TRUE 
    while (counter <= length(database) & cont == TRUE){
        data_pt <-database[[counter]]
        if (data_pt[1] == item_to_get){
            return(data_pt)
            cont <- FALSE
        }
        counter <- counter +1
    }
}

get_pts_range_dist <- function(lower_bound, higher_bound){
    cont = TRUE
    counter <-1
    eligible_vertices <-list()
    while (counter <= length(distance_based) & cont ==TRUE){
        distance_wrt_pivot <- distance_based[[counter]]
        
        if (distance_wrt_pivot[[2]] >= lower_bound & distance_wrt_pivot[[2]]<= higher_bound){
            eligible_vertices <- append(eligible_vertices, distance_wrt_pivot[[1]])
        }
        if (distance_wrt_pivot[[2]]> higher_bound){
            cont <- FALSE
        }
        counter <- counter +1
    }
    return(eligible_vertices)
}

obtain_angle_range <- function(lower_bound, higher_bound){
    cont = TRUE
    counter <-1
    eligible_vertices <-list()
    while (counter <= length(angle_based) & cont ==TRUE){
        angle_wrt_hor <- angle_based[[counter]]
        
        if (angle_wrt_hor[[3]] >= lower_bound & angle_wrt_hor[[3]]<= higher_bound){
            eligible_vertices <- append(eligible_vertices, angle_wrt_hor[[1]])
        }
        if (angle_wrt_hor[[3]]> higher_bound){
            cont <- FALSE
        }
        counter <- counter +1
    }
    return(eligible_vertices)
}

conform_angle <- function(angle){
    if (angle <0){
        angle <- 2*pi + angle
    }
    if (angle >2*pi){
        angle <- angle - 2*pi
    }
    
    return(angle)
}

get_pts_range_angle <- function(distance_from_pivot, radius, angle){
     eligible_vertices <-list()
    if (distance_from_pivot < 2*radius){
        for (point in 1:length(angle_based)){
            eligible_vertices <- append(eligible_vertices, angle_based[[point]][1])
            }
    }else{
        if (2*radius == distance_from_pivot){
            #r = d, gives the angle range that is 180 degrees. Gives which side?
            higher_bound <- conform_angle(pi/2 - (pi - angle))
            lower_bound<- conform_angle(higher_bound + pi)
            eligible_vertices <- append(eligible_vertices, obtain_angle_range(lower_bound, higher_bound))
        }else{
            #r > d
            phi <- asin(2*radius/distance_from_pivot)
            if (angle + phi > 2*pi){
                diff <-  angle + phi -2*pi 
                eligible_vertices <- append(eligible_vertices, obtain_angle_range(0,diff))
                eligible_vertices <- append(eligible_vertices, obtain_angle_range(angle-phi, 2*pi))
            } else {
                if (angle - phi < 0){
                    diff <- angle - phi 
                    eligible_vertices <- append(eligible_vertices, obtain_angle_range(2*pi + diff, 2*pi))
                    eligible_vertices <- append(eligible_vertices, obtain_angle_range(0, angle+phi))
                }else{
                    eligible_vertices <- append(eligible_vertices, obtain_angle_range(angle-phi, angle+phi))
                }
            }
        }
    }    
    return(eligible_vertices)

}


add_data_mat <- function(cluster1_pos,cluster2_pos,value, mat){
    position1 <- (cluster1_pos -1)*nrow(mat) + cluster2_pos
    position2 <- (cluster2_pos -1)*nrow(mat) + cluster1_pos
    mat[[position1]] <- value
    mat[[position2]] <- value
    return(mat)
}


choose_new_edge <- function(forward_edge, backward_edge, main){
    f_vertex <- forward_edge[[2]]
    b_vertex <- backward_edge[[1]]
    common <- forward_edge[[2]]
    compare <- current_tree[[main]]
    
    f_min <- Inf
    f_replace <- NULL
    b_min <- Inf
    b_replace <- NULL
    
    vPt_f <-values(pointSet, keys=f_vertex) 
    vPt_b <-values(pointSet, keys=b_vertex) 
    vPt_c <-values(pointSet, keys =common)
    
    for (vertex in compare){
        if (vertex != common){
            if (f_min == Inf){
                f_min <- dist_pts(vPt_f , vPt_c)
                b_min <- dist_pts(vPt_b, vPt_c)
                f_replace<-vertex
                b_replace<-vertex
            } else {
                dist_f <-dist_pts(vPt_f , vPt_c)
                if (dist_f < f_min){
                    f_min <- dist_f
                    f_replace <- vertex
                } 
                dist_b <- dist_pts(vPt_b , vPt_c)
                if (dist_b < b_min){
                    b_min <- dist_b
                    b_replace <- vertex
                }
            }  
        }
    }
    
    if (f_min <= b_min){
        return(list("f", f_replace))
    } else {
        return(list("b", b_replace))
    }
    
}

end_points_det <- function(cluster_tour, connections){
    merge <- hash()
    for (pt_index in 1:length(cluster_tour)){
        main <-cluster_tour[[pt_index]]
        
        if (pt_index == length(cluster_tour)){
            forward <- cluster_tour[[1]]
            backward <- cluster_tour[[pt_index - 1]]
            
        } else {
            if (pt_index == 1){
                forward <- cluster_tour[[pt_index+1]]
                backward <- cluster_tour[[length(cluster_tour)]]

                
            } else {
                forward <- cluster_tour[[pt_index+1]]
                backward <-cluster_tour[[pt_index-1]]

                }
        }
        forward_edge <- connections[[sprintf("%s-%s",main, forward)]]
        backward_edge <- connections[[sprintf("%s-%s",backward, main)]]
        print("forward_edge")
        print(forward_edge)
        print("backward_edge")
        print(backward_edge)
        
        if (forward_edge[[1]] == backward_edge[[2]]){
            print("connecting pts overlaps")
            if (length(current_tree[[main]]) != 1){
                print("branch length > 1, switching edge")
                compare_result <-  choose_new_edge(forward_edge, backward_edge, main)
                print("compare_result")
                print(compare_result)
                if (compare_result[[1]] == "f"){
                    forward_edge[[1]] <- compare_result[[2]]
                    
                } else {
                    backward_edge[[2]] <- compare_result[[2]]
                } 
            }else {print("its ok, only 1 pt in branch")}
        } 
        merge[[toString(main)]] <-list(backward_edge[[2]],forward_edge[[1]] )
        
        
        
    }
    return(merge)
}

add_end_points <- function(tsp, end_points){
    print(end_points)
    constant <- rep(c(Inf), nrow(as.matrix(tsp)) +1)
    constant[[end_points[[1]]]] <- 0
    constant[[end_points[[2]]]] <- 0
    tsp <-insert_dummy(tsp, 1, const =constant, inf = Inf, label = "dummy")
    return(tsp)
}


to_global <- function(tour_list, main){
        
        for (tour_index in 1:length(tour_list)){
                for (vertex_index in 1:length(tour_list[[tour_index]])){
                        vertex <- as.integer(tour_list[[tour_index]][vertex_index])
                       
                        tour_list[[tour_index]][vertex_index] <- main[[tour_index]][vertex]
                }
        }
        return(tour_list)
}

filter_before_merge <- function(tour_list, end_points_list){
    for (index in 1:length(tour_list)){
        sol <- tour_list[[index]]
        sol <- sol [lengths(sol) != 0]
        start <- end_points_list[[toString(index)]][[1]]
        end <- end_points_list[[toString(index)]][[2]]
        start_pos <- match(start, sol)

        end_pos <- match(end, sol)

        if (start_pos < end_pos){
            if (start_pos != 1 & end_pos != length(sol)){
                sol <- append(rev(sol[1:start_pos]), rev(sol[end_pos:length(sol)]))
                } 
        } else {
            
            if (start_pos == length(sol) & end_pos == 1){
                sol <- rev(sol)
            } else {
                sol <- append(rev(sol[1:end_pos]), rev(sol[start_pos:length(sol)]))
                sol <- rev(sol)
             }
        
        }
        tour_list[[index]] <- sol
    }
    return(tour_list)
} 

merge_all <- function(formatted_all_subtours, cluster_tour){
    merged_tour <- list()
    for (branch in cluster_tour){
        merged_tour <- append(merged_tour, all_subtours[[branch]])
    }
    print(as.integer(merged_tour))
    return(as.TOUR(as.integer(merged_tour)))
}


In [23]:
#main
library("TSP")
concorde_path("~/concorde/TSP")



dataTSP <- read_TSPLIB("/home/LC/mailo01/TSP-ML-Approach/src/R/CLUSTER_DIV/main.tsp")


found: concorde concorde.c concorde.o 


In [24]:
#main

library("hash")
library("sets")


radii <-seq(from=6.5, to=6.5, by=1)
processing_time <- list()
heuristic_tours <- list()

for (radius in radii){
    print(radius)
    start_time <- Sys.time()
    pointSet <- hash()
    noPts <- length(dataTSP)/2
    for (i in 1:noPts){
        pointSet[[toString(i)]] <- tuple(dataTSP[i], dataTSP[i+noPts])
    }

    pivot_point <- values(pointSet, keys =1)

    angle_based <- list()
    distance_based <-list()

    for (i in 1:noPts){
        compare_point <- values(pointSet, keys= i)
        element <-list(list(i, dist_pts(pivot_point,compare_point), angle_pts(compare_point,pivot_point)))
        angle_based <- append(angle_based, element)
        distance_based <- append(distance_based, element)
    }

    angle_based <- angle_based[order(sapply(distance_based, '[[', 3))]
    distance_based <- distance_based[order(sapply(distance_based,'[[',2))]

    
    current_tree <- list()
    checklist <-(c(1:noPts))
    while (length(checklist) != 0){

        branch <- list()

        old_branch_len<- length(branch)

        startPt = checklist[1]
        branch<-append(branch, startPt)

        counter =1

        while(old_branch_len != length(branch)){

            current_branch_len <- length(branch)

            start <-old_branch_len+1
            pts_to_connect <- c(branch[start:current_branch_len])


            checklist <-checklist[!checklist %in% pts_to_connect]

            old_branch_len= current_branch_len

            for (connect_pt in pts_to_connect){
                ## to yield an estimates of near by points 
                pivot_data <- get_item(connect_pt, distance_based)
                dist_wrt_pivot <- pivot_data[[2]][1]
                angle_wrt_hor <-pivot_data[[3]][1]


                if (dist_wrt_pivot == 0){
                    possible_connect <- get_pts_range_dist(0,2*radius)

                } else {

                    possible_angle <- get_pts_range_angle(dist_wrt_pivot, radius, angle_wrt_hor)

                    possible_dist <- get_pts_range_dist(dist_wrt_pivot-2*radius, dist_wrt_pivot+2*radius)

                    possible_connect <- intersect(possible_angle, possible_dist)
                }

                for (comparePt in possible_connect){

                    if (!comparePt %in%branch){
                        vPt1 <-values(pointSet, keys=connect_pt) 
                        vPt2 <-values(pointSet, keys=comparePt) 

                        if (dist_pts(vPt1,vPt2) <= 2*radius){

                            branch<-append(branch,comparePt)
                            checklist <-checklist[!checklist %in% c(comparePt)]


                        } else {}
                    } else{}
                }
            }
            counter <- counter +1

        }
        current_tree <- append(current_tree, list(branch))
    }

    branch_no <- length(current_tree)
    cluster_mat <- matrix(rep(0,branch_no^2),nrow=branch_no)

    #for each branch, need to find the shortest distance 
    #remember, have to put points already taken out of consideration

    connections <- hash()

    for (branch_index in 1:branch_no){
        #other branches to be compared to
            for (branch_compare_ind in 1:length(current_tree)){
                if (has.key(sprintf("%s-%s",branch_index,branch_compare_ind),connections) == FALSE){

                if (branch_index != branch_compare_ind){
                    minimum_dist <- Inf
                    connect_pts <- NULL

                    for (vertex in current_tree[[branch_index]]){

                        for (vertex_compare in current_tree[[branch_compare_ind]]){
                            vPt1 <-values(pointSet, keys=vertex) 
                            vPt2 <-values(pointSet, keys=vertex_compare) 
                            if (minimum_dist == Inf){
                                minimum_dist <- dist_pts(vPt1,vPt2)
                                connect_pts <- list(vertex, vertex_compare)
                            } else{
                                distance <- dist_pts(vPt1,vPt2)
                                if (distance <= minimum_dist){
                                    minimum_dist <- distance
                                    connect_pts <- list(vertex, vertex_compare)
                                } 


                            }
                        } 
                    }

                    connections[[sprintf("%s-%s",branch_index,branch_compare_ind)]] <-  tuple(connect_pts[[1]],connect_pts[[2]])
                    connections[[sprintf("%s-%s",branch_compare_ind,branch_index)]] <-  tuple(connect_pts[[2]],connect_pts[[1]])
                    #append distances to matrix
                    cluster_mat <- add_data_mat(branch_index,branch_compare_ind,minimum_dist, cluster_mat)

                }
            }
        }
    }

    tsp_cluster <- as.TSP(cluster_mat)
    cluster_tour <- solve_TSP(tsp_cluster, method="concorde")

    end_points_list<- end_points_det(cluster_tour, connections)
    print(end_points_list)
    all_solutions <- list()
    for (branchNo in 1:length(current_tree)){
        pts  <- data.frame(x=c(), y=c())
        if ( length(current_tree[[branchNo]]) <=2){
            if (length(current_tree[[branchNo]]) == 1){

                all_solutions <- append(all_solutions, list(list(1)))
            }

            if (length(current_tree[[branchNo]]) == 2){
                all_solutions <- append(all_solutions, list(list(2,1)))
            }

        }else{
            end_points <- end_points_list[[toString(branchNo)]]
            end_points_pos <-list()
            for (pt in current_tree[[branchNo]]){
                if (pt %in% end_points){
                    end_points_pos <- append(end_points_pos, match(pt,current_tree[[branchNo]] ))
                }
                pt_data <- values(pointSet, keys=pt)
                pts <- rbind(pts, c(pt_data[[1]],pt_data[2]))           

            }

            tsp_subcase <- TSP(dist(pts))
            end_points <- end_points_list[[toString(branchNo)]]

            tsp_subcase <- add_end_points(tsp_subcase, end_points_pos)
            tour <-  solve_TSP(tsp_subcase)
            all_solutions <- append(all_solutions, list(as.list(as.integer(tour))))
        }
    }

    to_global_subtours <- to_global(all_solutions, current_tree)
    all_subtours <- filter_before_merge(to_global_subtours,end_points_list)

    merged_tour <-merge_all(all_subtours, cluster_tour)
    end_time <- Sys.time()
    total_time <- as.numeric(end_time - start_time)
    processing_time<- append(processing_time, total_time)
    heuristic_tours <- append(heuristic_tours, merged_tour)
    

}

[1] 6.5
Used control parameters:
clo	 =  
exe	 =  /home/LC/mailo01/concorde/TSP/concorde
precision	 =  6
verbose	 =  TRUE
keep_files	 =  FALSE
[1] "forward_edge"
(1L, 19L)
[1] "backward_edge"
(20L, 1L)
[1] "connecting pts overlaps"
[1] "its ok, only 1 pt in branch"
[1] "forward_edge"
(19L, 14L)
[1] "backward_edge"
(1L, 19L)
[1] "connecting pts overlaps"
[1] "its ok, only 1 pt in branch"
[1] "forward_edge"
(17L, 11L)
[1] "backward_edge"
(19L, 14L)
[1] "forward_edge"
(11L, 2L)
[1] "backward_edge"
(17L, 11L)
[1] "connecting pts overlaps"
[1] "its ok, only 1 pt in branch"
[1] "forward_edge"
(2L, 12L)
[1] "backward_edge"
(11L, 2L)
[1] "connecting pts overlaps"
[1] "branch length > 1, switching edge"
[1] "compare_result"
[[1]]
[1] "f"

[[2]]
[1] 2

[1] "forward_edge"
(12L, 16L)
[1] "backward_edge"
(2L, 12L)
[1] "connecting pts overlaps"
[1] "its ok, only 1 pt in branch"
[1] "forward_edge"
(16L, 18L)
[1] "backward_edge"
(12L, 16L)
[1] "connecting pts overlaps"
[1] "its ok, only 1 pt in branch

ERROR: Error in end_points[[2]]: subscript out of bounds


In [15]:
current_tree

[[1]]
[[1]][[1]]
[1] 1


[[2]]
[[2]][[1]]
[1] 2

[[2]][[2]]
[1] 5

[[2]][[3]]
[1] 10


[[3]]
[[3]][[1]]
[1] 3

[[3]][[2]]
[1] 13

[[3]][[3]]
[1] 17

[[3]][[4]]
[1] 14

[[3]][[5]]
[1] 7


[[4]]
[[4]][[1]]
[1] 4


[[5]]
[[5]][[1]]
[1] 6


[[6]]
[[6]][[1]]
[1] 8

[[6]][[2]]
[1] 18


[[7]]
[[7]][[1]]
[1] 9


[[8]]
[[8]][[1]]
[1] 11


[[9]]
[[9]][[1]]
[1] 12


[[10]]
[[10]][[1]]
[1] 15


[[11]]
[[11]][[1]]
[1] 16


[[12]]
[[12]][[1]]
[1] 19


[[13]]
[[13]][[1]]
[1] 20

In [ ]:
plot(dataTSP)
symbols(dataTSP, circles=rep(radius,noPts), add=T, inches=F)